## Fit a linear model on cell morphology features

We aim to determine which features are significantly impacted by drug treatment, adjusted by cell count.

In [1]:
import pathlib
import pandas as pd

from sklearn.linear_model import LinearRegression

from pycytominer import feature_select
from pycytominer.cyto_utils import infer_cp_features

In [2]:
# Define inputs and outputs
plate = "localhost220513100001_KK22-05-198_FactinAdjusted"  # Focusing on plate 2
file_suffix = "_sc_norm_fs_cellprofiler.csv.gz"

data_dir = pathlib.Path("..", "..", "..", "3.process-cfret-features", "data")

cp_file = pathlib.Path(data_dir, f"{plate}{file_suffix}")

output_dir = pathlib.Path("results")
output_cp_file = pathlib.Path(output_dir, f"{plate}_linear_model_cp_features.tsv")

In [3]:
# Load data
cp_df = pd.read_csv(cp_file)

# Drop NA columns
cp_df = feature_select(
    cp_df,
    operation="drop_na_columns",
    na_cutoff=0
)

# Count number of cells per well and add to dataframe as metadata
cell_count_df = pd.DataFrame(
    cp_df.groupby("Metadata_Well").count()["Metadata_treatment"]
).reset_index()
cell_count_df.columns = ["Metadata_Well", "Metadata_cell_count_per_well"]
cp_df = cell_count_df.merge(cp_df, on=["Metadata_Well"])

# Clean the dose column to extract numeric value
cp_df = cp_df.assign(Metadata_dose_numeric=cp_df.Metadata_dose.str.strip("uM").astype(float))

# Define CellProfiler features
cp_features = infer_cp_features(cp_df)

print(f"We are testing {len(cp_features)} CellProfiler features")
print(cp_df.shape)
cp_df.head()

We are testing 585 CellProfiler features
(17352, 599)


,Metadata_Well,Metadata_cell_count_per_well,Metadata_WellRow,Metadata_WellCol,Metadata_heart_number,Metadata_treatment,Metadata_dose,Metadata_ImageNumber,Metadata_Plate,Metadata_Cytoplasm_Parent_Cells,...,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_03_256,Nuclei_Texture_SumEntropy_ER_3_03_256,Nuclei_Texture_SumEntropy_Golgi_3_01_256,Nuclei_Texture_SumEntropy_Mitochondria_3_03_256,Nuclei_Texture_SumVariance_Actin_3_01_256,Nuclei_Texture_SumVariance_ER_3_03_256,Nuclei_Texture_SumVariance_Golgi_3_01_256,Nuclei_Texture_SumVariance_Hoechst_3_01_256,Nuclei_Texture_SumVariance_Mitochondria_3_03_256,Metadata_dose_numeric
0,A09,342,A,9,9,drug_x,5uM,1,localhost220513100001,1,...,-0.771456,0.383014,1.527986,1.603349,-0.420076,-0.236242,0.203806,-0.354730,0.161636,5.0
1,A09,342,A,9,9,drug_x,5uM,1,localhost220513100001,2,...,0.965459,-0.922346,-0.491535,-1.000420,-0.460815,-0.468439,-0.206325,-0.331033,-0.189029,5.0
2,A09,342,A,9,9,drug_x,5uM,1,localhost220513100001,3,...,-1.411945,0.123190,0.934086,0.958949,-0.468958,-0.237587,-0.036968,-0.107819,-0.063200,5.0
3,A09,342,A,9,9,drug_x,5uM,1,localhost220513100001,4,...,-1.835825,0.908000,1.779381,1.798252,-0.463245,0.843474,0.579924,-0.257025,0.535400,5.0
4,A09,342,A,9,9,drug_x,5uM,1,localhost220513100001,5,...,0.001208,-0.555080,0.129196,-0.043079,-0.472035,-0.441913,-0.169445,-0.353311,-0.162409,5.0


## Fit linear model

In [4]:
# Setup linear modeling framework
variables = ["Metadata_cell_count_per_well", "Metadata_dose_numeric"]
X = cp_df.loc[:, variables]

print(X.shape)
X.head()

(17352, 2)


,Metadata_cell_count_per_well,Metadata_dose_numeric
0,342,5.0
1,342,5.0
2,342,5.0
3,342,5.0
4,342,5.0


In [5]:
# Fit linear model for each feature
lm_results = []
for cp_feature in cp_features:
    # Subset CP data to each individual feature (univariate test)
    cp_subset_df = cp_df.loc[:, cp_feature]

    # Fit linear model
    lm = LinearRegression(fit_intercept=True)
    lm_result = lm.fit(X=X, y=cp_subset_df)
    
    # Extract Beta coefficients
    # (contribution of feature to X covariates)
    coef = lm_result.coef_
    
    # Estimate fit (R^2)
    r2_score = lm.score(X=X, y=cp_subset_df)
    
    # Add results to a growing list
    lm_results.append([cp_feature, r2_score] + list(coef))

# Convert results to a pandas DataFrame
lm_results = pd.DataFrame(
    lm_results,
    columns=["feature", "r2_score", "cell_count_coef", "treatment_dose_coef"]
)

# Output file
lm_results.to_csv(output_cp_file, sep="\t", index=False)

print(lm_results.shape)
lm_results.head()

(585, 4)


,feature,r2_score,cell_count_coef,treatment_dose_coef
0,Cytoplasm_AreaShape_Compactness,0.044215,-0.000025,-0.078005
1,Cytoplasm_AreaShape_Extent,0.069271,-0.000077,0.096291
2,Cytoplasm_AreaShape_FormFactor,0.095343,-0.000160,0.111922
3,Cytoplasm_AreaShape_MajorAxisLength,0.083236,-0.001751,0.014357
4,Cytoplasm_AreaShape_Perimeter,0.045651,-0.001369,-0.011158
